---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="db6af3ab" -->
# Table des matières
- [Exemple de classification avec régression logistique](#exemple-de-classification-avec-régression-logistique)
  - [Lecture et préparation des données](#lecture-et-préparation-des-données)
      - [Lecture des données](#lecture-des-données)
      - [Séparation des données en ensembles d'entraînement et de test](#séparation-des-données-en-ensembles-dentraînement-et-de-test)
      - [Normalisation des données](#normalisation-des-données)
  - [Entraînement d'un classificateur de type régression logistique](#entraînement-dun-classificateur-de-type-régression-logistique)
      - [Sélection du type de classificateur.](#sélection-du-type-de-classificateur)
      - [Entraînement du classificateur avec les données d'entraînement](#entraînement-du-classificateur-avec-les-données-dentraînement)
      - [Normalisation des données de test](#normalisation-des-données-de-test)
      - [Prédiction des classes pour l'ensemble de test](#prédiction-des-classes-pour-lensemble-de-test)
      - [Affichage des statistiques de classification](#affichage-des-statistiques-de-classification)
  - [Interprétation des résultats](#interprétation-des-résultats)
<!-- #endregion -->

<!-- #region id="18d78b5d" -->
---
# Exemple de classification avec régression logistique
---
<!-- #endregion -->

<!-- #region id="24bea696" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/penguins1.png"  width="500" />
    <div>
    <font size="1.5">Image Source: https://github.com/allisonhorst/penguins/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="769f9103" -->
Dans cette section, nous allons appliquer la régression logistique afin de classifier trois espèces de manchots
(*penguins*) à partir de mensurations de becs. Puis, nous allons voir comment interpréter les résultats afin
de leur donner un sens autrement que numérique. C'est un type d'application en apprentissage automatique
couramment utilisée par les biologistes.

Nous allons utiliser le  jeu de données [penguin](https://www.kaggle.com/parulpandey/penguin-dataset-the-new-iris)
qui est disponible sur Scikit Learn.



La régression logistique modélise la probabilité d'appartenance à la classe 1
(plutôt qu'à la classe 0) comme suit

$$p(1\vert X, \Theta) = {\dfrac  {1}{1+e^{-(a_{0} + a_{1}x_{1} + \cdots +  a_{N}x_{N})}}}$$
où $\Theta=\{a_{0}, \cdots, a_{N}\}$ représente l'ensemble des paramètres du modèle et où les $x_{i}$ sont
 les variables, dans ce cas-ci, les mensurations de becs de manchots.

Puisque nous avons trois classes (Adélie, Chinstrap, Gentoo), et non deux, on traite d'abord le problème
comme trois problèmes indépendants:


- problème 1: p(Adélie|X),
- problème 2: p(Chinstrap|X),
- problème 3: p(Gentoo|X).


Puis, on sélectionne la classe ayant la plus grande probabilité. C'est l'approche "un contre tous"
(*one-versus-all*). Cette opération est faite de façon transparente par la fonction
__[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)__
dans la librairie Scikit-Learn. Pas besoin de faire trois classifications!
<!-- #endregion -->



In [ ]:
%matplotlib inline

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)

# Pour la reproductibilité des résultats
seed = 42
np.random.seed(seed)



<!-- #region id="6a10fb38" -->
## Lecture et préparation des données

Le  jeu de données contient sept variables (colonnes). La colonne 1 contient la
réponse que l'on veut prédire, soit l'espèce de manchot. Les colonnes 2 à 7 contiennent les variables mesurées
(quatre quantitatives et deux catégorielles).

<!-- #endregion -->

<!-- #region id="48329652" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/penguin-culmen.png"  width="400" />
    <div>
    <font size="1.5">Image Source: https://allisonhorst.github.io/palmerpenguins/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="a3ac700f" -->
Les variables quantitatives correspondent à la longueur et à l'épaisseur du bec (*bill*), à la longueur des
nageoires (*flipper*) et à la masse de chaque animal. Les variables catégorielles correpondent à l'île
où habitent les animaux et leur sexe.

#### Lecture des données
<!-- #endregion -->



In [ ]:
# Lecture du jeu de données en format CSV et écriture dans un DataFrame
df = sns.load_dataset("penguins")

# Affichage des cinq premières lignes du fichier.
df.head()



<!-- #region id="743b66a9" -->
On peut faire les observations suivantes:


- il y a quelques lignes contenant des valeurs manquantes (NaN),
- les valeurs des facteurs quantitatifs couvrent différent ordres de grandeur.


Élimination des lignes contenant des valeurs manquantes.
<!-- #endregion -->



In [ ]:
df.dropna(inplace=True)



<!-- #region id="3d2e41f4" -->
Nous allons également éliminer les variables suivantes afin de rendre la tâche plus difficile au classificateur:


- sexe,
- nom de l'île,
- longueur des nageoires,
- masse.


En effet, l'ensemble des variables permet de distinguer les manchots presque parfaitement. C'est la raison
pour laquelle elles ont été choisies par les biologistes.

Élimination des colonnes non utilisées.
<!-- #endregion -->



In [ ]:
df.drop(['island', 'sex', 'flipper_length_mm', 'body_mass_g'], inplace=True, axis=1)

# Affichage des cinq premières lignes de la base de donnnées résultante
df.head()



<!-- #region id="ff0c5348" -->
#### Séparation des données en ensembles d'entraînement et de test

Extraction des variables X et de la réponse y
<!-- #endregion -->



In [ ]:
X = df.drop(df.columns[[0]], axis=1)
y = df.drop(df.columns[[1, 2]], axis=1)

# Liste des variables utilisées
feature_list = list(X.columns)



<!-- #region id="55967f6e" -->
Génération des ensembles d'entraînement ($80~\%$ des données) et de test ($20~\%$ restants). On utilise un
échantillonnage stratifié afin de respecter la distribution des espèces de manchots dans chaque ensemble.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y, random_state=seed
)



<!-- #region id="7a44e028" -->
#### Normalisation des données

Les paramètres de la fonction de normalisation sont calculés à partir des données d'entraînement **uniquement**.
<!-- #endregion -->



In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)



<!-- #region id="cb337188" -->
## Entraînement d'un classificateur de type régression logistique

#### Sélection du type de classificateur
<!-- #endregion -->



In [ ]:
clf = LogisticRegression()



<!-- #region id="d1d8f153" -->
#### Entraînement du classificateur avec les données d'entraînement
<!-- #endregion -->



In [ ]:
clf.fit(X_train_s, np.array(y_train).ravel());



<!-- #region id="0d55667d" -->
#### Normalisation des données de test

Afin de mesurer les performances du modèle avec l'ensemble de test, il faut d'abord normaliser ces nouvelles données
avec la même transformation qui a été appliquée aux données d'entraînement.
<!-- #endregion -->



In [ ]:
X_test_s = scaler.transform(X_test)



<!-- #region id="2821eac1" -->
#### Prédiction des classes pour l'ensemble de test
<!-- #endregion -->



In [ ]:
pred = clf.predict(X_test_s)



<!-- #region id="84b0d35f" -->
#### Affichage des statistiques de classification

Affichage du rapport sur les statistiques de classification pour obtenir une comparaison plus fine des performances en test.

<!-- #endregion -->



In [ ]:
print(classification_report(y_test, pred))



<!-- #region id="5c061bbc" -->
On obtient une exactitude (*accuracy*) de $96~\%$. Ainsi $96~\%$ des prédictions d'espèces de manchots sont correctes.

On observe des F1-scores de $88~\%$ à $100~\%$; les manchots Chinstrap sont les plus difficiles à classifier.

La précision et le rappel s'expliquent comme suit. Dans le cas de l'espèce Chinstrap, $100~\%$ des prédictions Chinstrap étaient correctes. Par contre seulement $79~\%$ des manchots Chinstrap ont été identifiés.

Comment expliquer cette différence entre précision et rappel? Le classificateur reconnait les manchots Chinstrap les plus faciles à identifier. Il les classifie parfaitement; $100~\%$ de ses précisions sont correctes. Par contre, il ne reconnait pas les manchots Chinstrap un peu atypiques (très jeunes? très vieux?). Il en oublie plusieurs; il ne peut identifier que $79~\%$ de leur population. Cet exemple montre que les deux métriques de rappel et de précision sont complémentaires. Se fier uniquement à la précision est dangereux. Imaginez que vous avez conçu un détecteur de lions pour la maison et qu'il a une précision de $100~\%$ et un rappel de $10~\%$. À toutes les fois qu'il en détectera un, vous pourrez être certain que c'en est un. Par contre, le détecteur ne pourra détecter que $10~\%$ des lions dans la maison!
<!-- #endregion -->

<!-- #region id="a633a79d" -->
## Interprétation des résultats

<!-- #endregion -->

<!-- #region id="4fc1e1bc" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="1e39495f" -->
Nous avons conçu un bon classificateur. Voyons maintenant comment interpréter les paramètres des modèles.
La probabilité d'appartenance à une espèce de manchot est

$$p(\text{1}\vert X) = p(\text{Espèce}\vert x_{1}, x_{2}) = {\dfrac  {1}{1+e^{-(a_{0} + a_{1}x_{1} + a_{2}x_{2})}}}$$

On remarque que plus le terme $a_{1}x_{1} + a_{2}x_{2}$ dans l'équation est grand, plus la probabilité est grande.

Affichons les valeurs des $a_{i}$ pour chaque espèce de manchot.

<!-- #endregion -->



In [ ]:
print("Coefficients des modèles: \t %s \t %s \n" % (feature_list[0], feature_list[1]))
for i, espèce in enumerate(clf.classes_):
    print(
        "\t Espèce: %s \t a_{1} = %0.2f \t\t a_{2} = %0.2f\n"
        % (espèce, clf.coef_[i][0], clf.coef_[i][1])
    )



<!-- #region id="cd78a86c" -->
On remarque que pour les manchots Adélie, $a_{1}<0$ et $a_{2}>0$. Le terme $a_{1}x_{1}+a_{2}x_{2}$
devient maximal pour une petite valeur de $x_{1}$ et une grande valeur de $x_{2}$. Il est donc plus
probable d'observer un manchot Adélie avec un bec trapu (court et épais).

C'est l'inverse pour les manchots Gentoo, $a_{1}>0$ et $a_{2}<0$. Le terme $a_{1}x_{1}+a_{2}x_{2}$
devient maximal pour une grande valeur de $x_{1}$ et une petite valeur de $x_{2}$. Il est donc plus probable
d'observer un manchot Gentoo avec un bec effilé (long et mince).

Finalement, les manchots Chinstrap ont $a_{1}>0$ et $a_{2}>0$. Il est donc plus probable
de les observer avec un bec long et épais.

C'est bien ce que l'on observe dans la figure suivante.

<!-- #endregion -->

<!-- #region id="9df7ac16" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/penguins2.jpeg" width="500" />
    <div>
    <font size="0.5">Image Source: https://www.pulseheadlines.com/climate-change-is-directly-affecting-the-penguin-population/27828/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="76825aac" -->
Dans ce module, on a réussi à classifier les différentes espèces de manchots en plus d'expliquer les critères de
classification. Le classificateur n'est pas une boîte noire que les biologistes doivent utiliser. Au contraire, ses
critères de classification sont facilement compréhensibles en plus de montrer l'importance des variables
mesurées sur le terrain.
<!-- #endregion -->
